# 개체명 인식(Named Entity Recognition)


* 개체명 인식은 텍스트에서 이름을 가진 개체를 인식하는 기술      
* 가령, '철수와 영희는 밥을 먹었다'에서 이름과 사물을 추출하는 개체명 인식 모델 결과

  철수 - 이름    
  영희 - 이름    
  밥 - 사물

## 개체명 인식 - NLTK

* https://wikidocs.net/30682

* `nltk` 라이브러리에서는 미리 학습된 개체명 인식 모델을 제공

### 라이브러리 준비

In [2]:
import nltk

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

### 토큰화 및 품사 태깅

In [3]:
from nltk import word_tokenize, pos_tag, ne_chunk

sentence = 'James is working at Disney in London'
sentence = pos_tag(word_tokenize(sentence))
print(sentence)

[('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('Disney', 'NNP'), ('in', 'IN'), ('London', 'NNP')]


### 개체명 인식

In [4]:
sentence = ne_chunk(sentence)
print(sentence)

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Disney/NNP)
  in/IN
  (GPE London/NNP))


## 개체명 인식 - LSTM

* https://wikidocs.net/24682

* 사용자가 제공되고 있는 개체명 인식 모델과는 다른 개체명을 정의해 사용하는 것이 필요할 수 있음
* 직접 개체명 인식 모델을 구성해 학습하고 사용할 수 있음

### 라이브러리 준비

In [5]:
import numpy as np
import urllib.request

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### 데이터 준비

* 공개된 개체명 인식 데이터셋을 이용
  + https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt
* 해당 데이터는 단어-개체명 형식으로 이루어져 있으므로 이를 가공해 데이터셋을 생성

In [6]:
tagged_sentences = []
sentence = []

with urllib.request.urlopen('https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt') as f:
  for line in f:
    line = line.decode('utf-8')
    if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == '\n': # 문장 구분
      if len(sentence) > 0:
        tagged_sentences.append(sentence)
        sentence = []
      continue
    splits = line.strip().split(' ')
    word = splits[0].lower()
    sentence.append([word, splits[-1]])

print(len(tagged_sentences))
print(tagged_sentences[0]) # 개체명 있는 것들 확인 가능

14041
[['eu', 'B-ORG'], ['rejects', 'O'], ['german', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['british', 'B-MISC'], ['lamb', 'O'], ['.', 'O']]


### 데이터 전처리

* 단어와 개체명 태그를 분리해서 데이터를 구성

In [7]:
sentences, ner_tags = [], []

for tagged_sentence in tagged_sentences:
  sentence, tag_info = zip(*tagged_sentence)
  sentences.append(list(sentence))
  ner_tags.append(list(tag_info))

* 정제 및 빈도 수가 높은 상위 단어들만 추출하기 위해 토큰화 작업

In [8]:
max_words = 4000
src_tokenizer = Tokenizer(num_words=max_words, oov_token='OOV')
src_tokenizer.fit_on_texts(sentences)

tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(ner_tags)

In [9]:
vocab_size = max_words
tag_size = len(tar_tokenizer.word_index) + 1

print(vocab_size)
print(tag_size)

4000
10


* 데이터를 학습에 활용하기 위해 데이터를 배열로 변환
* 해당 작업은 토큰화 툴의 `texts_to_sequences()`를 통해 수행

In [10]:
X_train = src_tokenizer.texts_to_sequences(sentences)
y_train = tar_tokenizer.texts_to_sequences(ner_tags)

* 학습에 투입할 때는 동일한 길이를 가져야 하므로, 지정해둔 최대 길이에 맞춰 모든 데이터를 동일한 길이로 맞춰줌
* 일반적으로 길이를 맞출 때는 모자란 길이만큼 0을 추가


In [11]:
max_len = 70
X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
y_train = pad_sequences(y_train, padding='post', maxlen=max_len)

* 훈련, 실험 데이터 분리 및 원 핫 인코딩을 시행

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.2, random_state=111)

y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

* 최종적으로 생성된 데이터셋의 크기는 다음과 같음

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11232, 70)
(11232, 70, 10)
(2809, 70)
(2809, 70, 10)


### 모델 구축 및 학습

* 모델 구축에는 `keras`를 이용
* 해당 작업에 필요한 함수들을 추가로 import

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from keras.optimizers import Adam

모델의 구성

1. 입력을 실수 벡터로 임베딩
2. 양방향 LSTM 구성
3. Dense layer를 통한 각 태그에 속할 확률 예측

`TimeDistributed`는 상위 layer의 출력이 step에 따라 여러 개로 출력되어 이를 적절하게 분배해주는 역할

In [15]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len, mask_zero=True))
# input_dim: 입력 데이터의 단어 집합(vocabulary)의 크기
# output_dim: 임베딩 벡터의 크기 ,이 값은 임베딩 공간의 차원을 나타냄
# input_length: 입력 시퀀스의 길이, 이 경우에는 입력 문장의 최대 길이인 max_len으로 설정됩니다.
# mask_zero: True로 설정하면 입력 값이 0인 경우 해당 값은 무시됩니다. 패딩(padding) 값이 0으로 지정되어 있을 경우 유용
model.add(Bidirectional(LSTM(256, return_sequences=True)))
# Bidirectional LSTM은 입력 시퀀스를 앞뒤로 두 번 처리하는 LSTM 구조입니다.
# 첫 번째는 입력 시퀀스를 원래 방향대로 처리하고, 두 번째는 역방향으로 처리합니다.
#이렇게 함으로써 모델은 현재 시점에 대한 출력을 계산할 때 이전과 이후의 모든 정보를 고려할 수 있습니다.
#이는 문맥을 더 잘 이해하고, 시퀀스 데이터의 양 끝에서 발생하는 정보도 잘 처리할 수 있게 해줍니다.
# 따라서 Bidirectional LSTM은 다양한 시퀀스 데이터 처리 작업에 유용하게 사용됩니다. 특히 자연어 처리(Natural Language Processing) 분야에서 문장을 이해하거나 번역하는 등의 작업에 널리 사용
# return_sequences: True로 설정하면 출력 시퀀스의 전체 시퀀스를 반환합니다. 이 경우 각 타임스텝에 대한 출력을 얻기 위해 True로 설정
model.add(TimeDistributed(Dense(tag_size, activation='softmax'))) # 다중분류
# TimeDistributed
# 레이어는 다른 레이어를 래핑하여 시간 분배된(time-distributed) 입력을 처리하는 데 사용
# 시간 축을 따라 같은 레이어의 가중치를 공유합니다. 이것은 모델이 시간적인 정보를 고려하여 입력을 처리할 수 있도록 도와줍
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 70, 128)           512000    
                                                                 
 bidirectional (Bidirection  (None, 70, 512)           788480    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 70, 10)            5130      
 ributed)                                                        
                                                                 
Total params: 1305610 (4.98 MB)
Trainable params: 1305610 (4.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


* 모델 컴파일 및 학습 진행, 평가

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              # Adam은 경사 하강법의 한 종류로, 자동으로 학습률을 조절하면서 손실 함수를 최소화합니다.
              # 학습률은 0.001로 설정
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=3, validation_data=(X_test, y_test))

Epoch 1/3
88/88 [==============================] - 27s 160ms/step - loss: 0.8973 - accuracy: 0.8234 - val_loss: 0.6045 - val_accuracy: 0.8336
Epoch 2/3
88/88 [==============================] - 6s 73ms/step - loss: 0.4677 - accuracy: 0.8593 - val_loss: 0.3636 - val_accuracy: 0.8953
Epoch 3/3
88/88 [==============================] - 3s 39ms/step - loss: 0.3027 - accuracy: 0.9105 - val_loss: 0.2494 - val_accuracy: 0.9269


In [18]:
model.evaluate(X_test, y_test)

88/88 [==============================] - 1s 7ms/step - loss: 0.2514 - accuracy: 0.9269


[0.25140300393104553, 0.926934540271759]

### 학습한 모델을 통한 예측

* 예측을 확인하기 위해서 인덱스를 단어로 변환해줄 사전이 필요
* 사전은 토큰화 툴의 사전을 이용

In [19]:
idx2word = src_tokenizer.index_word
idx2ner = tar_tokenizer.index_word
idx2ner[0] = 'PAD'

* 예측 시각화

In [28]:
i = 10 # 인덱스
y_predicted = model.predict(np.array([X_test[i]]))
y_predicted = np.argmax(y_predicted, axis=-1)
true = np.argmax(y_test[i], -1)

print('{:15}|{:5}|{}'.format('단어','실제값','예측값'))
print('-' * 34)

for w, t, pred in zip(X_test[i], true, y_predicted[0]):
  if w != 0:
    print('{:17} {:7} {}'.format(idx2word[w], idx2ner[t].upper(), idx2ner[pred].upper()))


1/1 [==============================] - 0s 20ms/step
단어             |실제값  |예측값
----------------------------------
OOV               B-PER   B-PER
ballanger         I-PER   B-LOC
of                O       O
france            B-LOC   B-LOC
confirmed         O       O
her               O       O
status            O       O
as                O       O
the               O       O
world             O       O
's                O       O
number            O       O
one               O       O
woman             O       O
OOV               O       O
when              O       O
she               O       O
OOV               O       O
her               O       O
title             O       O
at                O       O
the               O       O
world             O       O
cycling           O       O
championships     O       O
on                O       O
friday            O       O
.                 O       O
